In [29]:
import pandas as pd
import numpy as np
import altair as alt
import nltk
import re
import string

In [30]:
from sklearn.cluster import KMeans
import gensim.corpora as corpora
from gensim.sklearn_api import TfIdfTransformer
from gensim.matutils import corpus2dense, corpus2csc

In [31]:
df = pd.read_csv('./IEEE VIS papers 1990-2018 - Main dataset.csv')

# Data Preprocessing and Cleaning

In [32]:
# Remove missing values

df = df.dropna(subset=['Abstract'])

In [33]:
# Turn text into lowercase

df['Abstract'] = [abstract.lower() for abstract in df['Abstract']]
df['Abstract'] = [abstract[:-19] for abstract in df['Abstract']]
df.iloc[0]['Abstract']

'the use of critical point analysis to generate representations of the vector field topology of numerical flow data sets is discussed. critical points are located and characterized in a two-dimensional domain, which may be either a two-dimensional flow field or the tangential velocity field near a three-dimensional body. tangent curves are then integrated out along the principal directions of certain classes of critical points. the points and curves are linked to form a skeleton representing the two-dimensional vector field topology. when generated from the tangential velocity field near a body in a three-dimensional flow, the skeleton includes the critical points and curves which provide a basis for analyzing the three-dimensional structure of the flow separation.'

In [34]:
# Remove punctuation

def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['Abstract'] = df['Abstract'].apply(lambda x: remove_punct(x))
df.iloc[0]['Abstract']

'the use of critical point analysis to generate representations of the vector field topology of numerical flow data sets is discussed critical points are located and characterized in a twodimensional domain which may be either a twodimensional flow field or the tangential velocity field near a threedimensional body tangent curves are then integrated out along the principal directions of certain classes of critical points the points and curves are linked to form a skeleton representing the twodimensional vector field topology when generated from the tangential velocity field near a body in a threedimensional flow the skeleton includes the critical points and curves which provide a basis for analyzing the threedimensional structure of the flow separation'

In [35]:
# Tokenize text so that the words in text are separated by commas

def tokenize(text):
    tokens = re.split("\W+", text)
    return tokens

df['Abstract'] = df['Abstract'].apply(lambda x: tokenize(x))
df.iloc[0]['Abstract']

['the',
 'use',
 'of',
 'critical',
 'point',
 'analysis',
 'to',
 'generate',
 'representations',
 'of',
 'the',
 'vector',
 'field',
 'topology',
 'of',
 'numerical',
 'flow',
 'data',
 'sets',
 'is',
 'discussed',
 'critical',
 'points',
 'are',
 'located',
 'and',
 'characterized',
 'in',
 'a',
 'twodimensional',
 'domain',
 'which',
 'may',
 'be',
 'either',
 'a',
 'twodimensional',
 'flow',
 'field',
 'or',
 'the',
 'tangential',
 'velocity',
 'field',
 'near',
 'a',
 'threedimensional',
 'body',
 'tangent',
 'curves',
 'are',
 'then',
 'integrated',
 'out',
 'along',
 'the',
 'principal',
 'directions',
 'of',
 'certain',
 'classes',
 'of',
 'critical',
 'points',
 'the',
 'points',
 'and',
 'curves',
 'are',
 'linked',
 'to',
 'form',
 'a',
 'skeleton',
 'representing',
 'the',
 'twodimensional',
 'vector',
 'field',
 'topology',
 'when',
 'generated',
 'from',
 'the',
 'tangential',
 'velocity',
 'field',
 'near',
 'a',
 'body',
 'in',
 'a',
 'threedimensional',
 'flow',
 'the

In [36]:
# Remove stopwords (words that add little value to sentence)

stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords]
    return text

df['Abstract'] = df['Abstract'].apply(lambda x: remove_stopwords(x))
df.iloc[0]['Abstract']

['use',
 'critical',
 'point',
 'analysis',
 'generate',
 'representations',
 'vector',
 'field',
 'topology',
 'numerical',
 'flow',
 'data',
 'sets',
 'discussed',
 'critical',
 'points',
 'located',
 'characterized',
 'twodimensional',
 'domain',
 'may',
 'either',
 'twodimensional',
 'flow',
 'field',
 'tangential',
 'velocity',
 'field',
 'near',
 'threedimensional',
 'body',
 'tangent',
 'curves',
 'integrated',
 'along',
 'principal',
 'directions',
 'certain',
 'classes',
 'critical',
 'points',
 'points',
 'curves',
 'linked',
 'form',
 'skeleton',
 'representing',
 'twodimensional',
 'vector',
 'field',
 'topology',
 'generated',
 'tangential',
 'velocity',
 'field',
 'near',
 'body',
 'threedimensional',
 'flow',
 'skeleton',
 'includes',
 'critical',
 'points',
 'curves',
 'provide',
 'basis',
 'analyzing',
 'threedimensional',
 'structure',
 'flow',
 'separation']

In [39]:
# Lemmatize words

wn = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

df['Abstract'] = df['Abstract'].apply(lambda x: lemmatizing(x))
df.iloc[0]['Abstract'] 

['use',
 'critic',
 'point',
 'analysi',
 'gener',
 'represent',
 'vector',
 'field',
 'topolog',
 'numer',
 'flow',
 'data',
 'set',
 'discus',
 'critic',
 'point',
 'locat',
 'character',
 'twodimension',
 'domain',
 'may',
 'either',
 'twodimension',
 'flow',
 'field',
 'tangenti',
 'veloc',
 'field',
 'near',
 'threedimension',
 'bodi',
 'tangent',
 'curv',
 'integr',
 'along',
 'princip',
 'direct',
 'certain',
 'class',
 'critic',
 'point',
 'point',
 'curv',
 'link',
 'form',
 'skeleton',
 'repres',
 'twodimension',
 'vector',
 'field',
 'topolog',
 'gener',
 'tangenti',
 'veloc',
 'field',
 'near',
 'bodi',
 'threedimension',
 'flow',
 'skeleton',
 'includ',
 'critic',
 'point',
 'curv',
 'provid',
 'basi',
 'analyz',
 'threedimension',
 'structur',
 'flow',
 'separ']

In [40]:
# Stem words

ps = nltk.PorterStemmer()

def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

df['Abstract'] = df['Abstract'].apply(lambda x: stemming(x))
df.iloc[0]['Abstract']

['use',
 'critic',
 'point',
 'analysi',
 'gener',
 'repres',
 'vector',
 'field',
 'topolog',
 'numer',
 'flow',
 'data',
 'set',
 'discu',
 'critic',
 'point',
 'locat',
 'charact',
 'twodimens',
 'domain',
 'may',
 'either',
 'twodimens',
 'flow',
 'field',
 'tangenti',
 'veloc',
 'field',
 'near',
 'threedimens',
 'bodi',
 'tangent',
 'curv',
 'integr',
 'along',
 'princip',
 'direct',
 'certain',
 'class',
 'critic',
 'point',
 'point',
 'curv',
 'link',
 'form',
 'skeleton',
 'repr',
 'twodimens',
 'vector',
 'field',
 'topolog',
 'gener',
 'tangenti',
 'veloc',
 'field',
 'near',
 'bodi',
 'threedimens',
 'flow',
 'skeleton',
 'includ',
 'critic',
 'point',
 'curv',
 'provid',
 'basi',
 'analyz',
 'threedimens',
 'structur',
 'flow',
 'separ']

# Exploratory Data Analysis (EDA)

### How many abstracts were presented each year?

In [14]:
to_plot1 = df.groupby('Year').count().reset_index()
alt.Chart(to_plot1).mark_line().encode(
    x = "Year:Q",
    y = "Title:Q"
)

alt.Chart(...)

### How many total papers are in each track?

In [15]:
to_plot2 = df.groupby('Conference').count().reset_index()

alt.Chart(to_plot2).mark_bar().encode(
    x = "Conference:N",
    y = "Abstract:Q",
)

alt.Chart(...)

### How many abstracts were presented by different tracks each year?

In [16]:
to_plot3 = df.groupby(['Conference', 'Year']).count().reset_index()
alt.Chart(to_plot3,height=150).mark_line().encode(
    x = "Year:Q",
    y = "Title:Q",
    color = 'Conference:N'
)

alt.Chart(...)

# TF-IDF Values

We generate tf-idf values for all the words in the abstract to evaluate how relevant a word is to the document.

In [20]:
def get_tfidf(input_data):
    # Create Dictionary
    id2word = corpora.Dictionary(input_data)
    model = TfIdfTransformer(dictionary=id2word)
    
    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in input_data]
    num_docs = id2word.num_docs
    num_terms = len(id2word.keys())
    
    # Get tfidf matrix
    tfidf_corpus = model.fit_transform(corpus)
    corpus_tfidf_dense = corpus2dense(tfidf_corpus, num_terms, num_docs)
    
    # Map index back to word
    words = []
    for i in id2word.keys():
        words.append(id2word[i])
    mat = pd.DataFrame(data=corpus_tfidf_dense, index=words)
    wordtfidf = pd.DataFrame(data=mat.values.T, columns=mat.index)
    return wordtfidf

# Clustering

We cluster the words based on their tf-idf values to group words into four different clusters.

In [21]:
tfidf_df = pd.DataFrame(get_tfidf(df['Abstract']))

In [22]:
cluster_model = KMeans(n_clusters=4)
cluster = cluster_model.fit(tfidf_df)
tfidf_cluster_df = pd.DataFrame(tfidf_df)

In [23]:
# top 10 words for each cluster

group_key_words = []
for label in range(4):
    # get the sum tf-idf for each word, do sum() across rows for each column
    group_df = tfidf_cluster_df[cluster.labels_ == label].mean(axis=0)
    # sort the tf-idf values
    to_sort = [{'freq': group_df[x], 'word': x} for x in group_df.index]
    to_sort = sorted(to_sort, key=lambda d: d['freq'], reverse=True)
    # add the words to the list
    for i in range(10):
        group_key_words.append([label, to_sort[i]['word'], to_sort[i]['freq']])
keyword_df = pd.DataFrame(data=group_key_words, columns=['label', 'keyword', 'tfidf'])

In [24]:
keyword_df

,label,keyword,tfidf
0,0,volum,0.054001
1,0,render,0.052114
2,0,surfac,0.051978
3,0,mesh,0.037842
4,0,algorithm,0.032967
5,0,isosurfac,0.028565
6,0,function,0.026370
7,0,imag,0.026103
8,0,method,0.024244
9,0,textur,0.023566


In [41]:
chart = alt.hconcat()

for label in range(4):
    chart |= alt.Chart(keyword_df[keyword_df['label']==label]).mark_bar().encode(
        x=alt.X('tfidf:Q', scale=alt.Scale(domain=[0,0.12]),title='cluster'+str(label)),
        y=alt.Y('keyword:N', sort='-x')
    ).properties(
        width = 50
    )
chart

alt.HConcatChart(...)

# Track Topics

We create four documents for each track and add all the words from papers in the track to each document. After, we get tf-idf values for all the words.

In [12]:
# Create four documents for each track and add all words from papers in the track to each document

track_key_words = []
tracks = ['Vis', 'InfoVis', 'VAST', 'SciVis']

for track in tracks:
    document = []
    track_index = df[df['Conference']==track].index.values.astype(int)
    for i in track_index:
        document.extend(df['Abstract'][i])
    track_key_words.append(document)

In [13]:
track_key_words

[['use',
  'critic',
  'point',
  'analysi',
  'gener',
  'represent',
  'vector',
  'field',
  'topolog',
  'numer',
  'flow',
  'data',
  'set',
  'discuss',
  'critic',
  'point',
  'locat',
  'character',
  'twodimension',
  'domain',
  'may',
  'either',
  'twodimension',
  'flow',
  'field',
  'tangenti',
  'veloc',
  'field',
  'near',
  'threedimension',
  'bodi',
  'tangent',
  'curv',
  'integr',
  'along',
  'princip',
  'direct',
  'certain',
  'class',
  'critic',
  'point',
  'point',
  'curv',
  'link',
  'form',
  'skeleton',
  'repres',
  'twodimension',
  'vector',
  'field',
  'topolog',
  'gener',
  'tangenti',
  'veloc',
  'field',
  'near',
  'bodi',
  'threedimension',
  'flow',
  'skeleton',
  'includ',
  'critic',
  'point',
  'curv',
  'provid',
  'basi',
  'analyz',
  'threedimension',
  'structur',
  'flow',
  'separ',
  'author',
  'discuss',
  'fast',
  'flow',
  'analysi',
  'softwar',
  'toolkit',
  'implement',
  'softwar',
  'system',
  'fluid',
  'mec

In [14]:
track_tfidf = get_tfidf(track_key_words)

In [15]:
track_tfidf

,,0,02,043spl,05,055,1,10,100,1000,...,wwwinviwoorg,wysiwyp,xct,xlinkhref24tvcg01jallepalli2744058ieq1sourcetif,xlinkhref24tvcg01zhang2744038ieq1sourcetif,xlinkhref24tvcg01zhang2744038ieq2sourcetif,xmlnsxlinkhttpwwww3org1999xlinkalternativesinlineformulacontinu,xmlnsxlinkhttpwwww3org1999xlinkalternativesinlineformulato1,xmlnsxlinkhttpwwww3org1999xlinkalternativesinlineformulaway,ϵsimplif
0,0.0,0.003749,0.003749,0.003749,0.001875,0.003749,0.0,0.0,0.0,0.001556,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.004455,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.007396,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.004185,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.005862,...,0.014125,0.02825,0.056501,0.014125,0.014125,0.014125,0.014125,0.014125,0.014125,0.014125


In [16]:
track_tfidf['Track'] = tracks

# Visualization

After grouping the words into the different tracks and retrieving their tf-idf values, we find the top ten words in each track by the words with the highest tf-idf values in each track.

In [17]:
# Top 10 words for each track

group_key_words = []
for track in tracks:
    # get the sum tf-idf for each word, do sum() across rows for each column
    group_df = track_tfidf[track_tfidf['Track'] == track].mean(axis=0)
    # sort the tf-idf values
    to_sort = [{'freq': group_df[x], 'word': x} for x in group_df.index]
    to_sort = sorted(to_sort, key=lambda d: d['freq'], reverse=True)
    # add the words to the list
    for i in range(10):
        group_key_words.append([track, to_sort[i]['word'], to_sort[i]['freq']])
keyword_df = pd.DataFrame(data=group_key_words, columns=['track', 'keyword', 'tfidf'])

In [18]:
keyword_df

,track,keyword,tfidf
0,Vis,isosurfac,0.523001
1,Vis,textur,0.245074
2,Vis,ray,0.234320
3,Vis,spl,0.172459
4,Vis,tetrahedr,0.127470
5,Vis,vortex,0.125595
6,Vis,projector,0.121846
7,Vis,lod,0.108724
8,Vis,unsteadi,0.097477
9,Vis,viewdepend,0.097477


In [19]:
chart = alt.hconcat()

for track in tracks:
    chart |= alt.Chart(keyword_df[keyword_df['track']== track], height=150).mark_bar().encode(
    x=alt.X('tfidf:Q', scale=alt.Scale(domain=[0,0.53]),title=track),
    y=alt.Y('keyword:N', sort='-x')
).properties(
    width = 50
)
chart

alt.HConcatChart(...)